In [ ]:
import os
import sys

def set_root_path():
    if os.getcwd().endswith('figures'):
        os.chdir('../')
set_root_path()
sys.path.append('python/')

In [ ]:
from pathlib import Path
import mitsuba as mi

mi.set_variant('cuda_ad_rgb')

from practical_reconstruction import optimization_cli
from core import integrators
from core import bsdfs
from core import textures

integrators.register()
bsdfs.register()
textures.register()

In [ ]:
import drjit as dr

def format_float(f):
  """Formats a float such that 0.1 becomes "0_1", 10.0 becomes "10_0", etc."""
  return str(f).replace('.', '_')

def l1_error(ref,img):
  return dr.mean(dr.abs(ref-img)).array[0]

def l2_error(ref,img):
  return dr.mean(dr.square(ref-img)).array[0]

def rel_l2_error(ref,img):
  numerator = dr.square(img - ref)
  denominator = (
      dr.square(img) + dr.square(ref) + 1e-2
  )
  loss = numerator / denominator
  return dr.mean(loss, axis=None).array[0]

In [ ]:
scene_name = 'art_gallery'
techniques = ['naive','naive_scales','mipmap','mipmap_scales','mipmap_pyramid']

# To accomodate lower end gpus we replaced the original 8K albedo map for a 4K map.
# You can replace `The_Great_Wave_off_Kanagawa_4k.001.jpg` with `The_Great_Wave_off_Kanagawa_8k.001.jpg`
# in the scene's texture folder to use the 8K map instead.

skip_existing = True

technique_configs = {
    'naive': [{'lr': 0.75}],
    'naive_scales': [{'lr': 0.05}],
    'mipmap': [{'lr': 1.0}],
    'mipmap_scales': [{'lr': 0.1}],
    'mipmap_pyramid': [{'lr': 0.0075}],
}

for technique in techniques:
    technique_params = technique_configs[technique]

    for technique_param in technique_params:
        base_learning_rate = technique_param['lr'] 

        print(
            f'******** Running {technique} with base learning rate'
            f' {base_learning_rate} ********'
        )

        override_bindings = []
        result_folder = f'results/{scene_name}/{technique}'

        result_folder += f'_lr_{format_float(base_learning_rate)}'
        override_bindings.append(
            f'SceneConfig.base_learning_rate={base_learning_rate}'
        )

        # Ensure that the default tmp folder is used
        override_bindings.append("SceneConfig.tmp_folder=''")

        override_bindings.append(
            f"SceneConfig.result_folder='{result_folder}'"
        )

        gin_config_name = f'{scene_name}/{technique}'

        print(f'Next result location: {result_folder}')
        if skip_existing and Path(result_folder).exists():
            print('Skipping, already present')
            continue

        # Run the config
        optimization_cli.run_config(gin_config_name, override_bindings)

## Figure after executing above

In [ ]:
from core import image_util
from core import mitsuba_io

from practical_reconstruction import figutils

import drjit as dr
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

In [ ]:
FIGURE_DIR = "figures/pdfs"
FIGURE_NAME = "mipmap_pyramid_motivation"

scene_name = "art_gallery"
scene_ref_name = scene_name

end_iter = 127
def load_images_and_errors():
  images = []
  res_suffix = ""
  res_suffix_ref = ""
  for i,iter in enumerate([end_iter,end_iter]):
    if i == 1:
      spp_suffix = "_spp_1024"
    else:
      spp_suffix = ""
    images.append([
        f"results/{scene_name}/naive_lr_0_75/frames/Camera.001_iter_{iter:03d}{res_suffix}{spp_suffix}.exr",
        f"results/{scene_name}/naive_scales_lr_0_05/frames/Camera.001_iter_{iter:03d}{res_suffix}{spp_suffix}.exr",
        f"results/{scene_name}/mipmap_lr_1_0/frames/Camera.001_iter_{iter:03d}{res_suffix}{spp_suffix}.exr",
        f"results/{scene_name}/mipmap_scales_lr_0_1/frames/Camera.001_iter_{iter:03d}{res_suffix}{spp_suffix}.exr",
        f"results/{scene_name}/mipmap_pyramid_lr_0_0075/frames/Camera.001_iter_{iter:03d}{res_suffix}{spp_suffix}.exr",
        f"third_party/{scene_ref_name}/references/ref_view_000{res_suffix_ref}.exr",
    ])

  boost = dr.sqrt(2)
  image_exr_paths_iter_start,image_exr_paths_iter_end = images
  num_images = len(image_exr_paths_iter_end)
  assert num_images == len(image_exr_paths_iter_start)
  images_exr_start, images_exr_end = [], []
  images_start, images_end = [], []
  for col in range(num_images):
    exr_start = mitsuba_io.read_bitmap(image_exr_paths_iter_start[col])
    exr_end = mitsuba_io.read_bitmap(image_exr_paths_iter_end[col])
    tonemapped_start = image_util.tonemap(boost * mi.TensorXf(exr_start.convert(
        pixel_format=mi.Bitmap.PixelFormat.RGB,
        component_format=mi.Struct.Type.Float32,
    )))
    tonemapped_end = image_util.tonemap(boost * mi.TensorXf(exr_end.convert(
        pixel_format=mi.Bitmap.PixelFormat.RGB,
        component_format=mi.Struct.Type.Float32,
    )))
    images_exr_start.append(mi.TensorXf(exr_start))
    images_exr_end.append(mi.TensorXf(exr_end))
    images_start.append(np.array(tonemapped_start))
    images_end.append(np.array(tonemapped_end))

  errors_start, errors_end = [],[]
  errors_start_l1, errors_end_l1 = [],[]
  ref_end = images_exr_end[-1]
  ref_start = images_exr_start[-1]
  for col in range(0, num_images-1):
    img = images_exr_end[col]
    error = l2_error(ref_end, img)
    errors_end.append(error)
    error_l1 = l1_error(ref_end,img)
    errors_end_l1.append(error_l1)

    img = images_exr_start[col]
    error = l2_error(ref_start, img)
    errors_start.append(error)
    error_l1 = l1_error(ref_start,img)
    errors_start_l1.append(error_l1)

  return images_start, images_end, errors_start, errors_end,errors_start_l1, errors_end_l1


In [ ]:
import matplotlib.gridspec as gridspec


def figure_grid_setup(
    fig_width, image_shape, image_crop_shape, inner_space=0.0, outer_space=0.1
):
  h, w = image_shape
  h_crop, w_crop = image_crop_shape
  r = w / h
  r_crop = w_crop / h_crop

  inner_wspace = inner_space
  inner_hspace = inner_wspace * r_crop
  inner_rows = 3
  inner_cols = 6
  inner_aspect = figutils.gridspec_aspect(
      n_rows=inner_rows,
      n_cols=inner_cols,
      w=w_crop,
      h=h_crop,
      wspace=inner_wspace,
      hspace=inner_hspace,
  )
  # Spacing in the main griddpec
  outer_rows = 1
  outer_cols = 2
  outer_wspace = outer_space
  outer_hspace = outer_space

  outer_aspect = figutils.gridspec_aspect(
      n_rows=outer_rows,
      n_cols=outer_cols,
      w=[r, inner_aspect],
      h=1,
      wspace=outer_wspace,
      hspace=outer_hspace,
  )

  fig = plt.figure(1, figsize=(fig_width, fig_width / outer_aspect))

  outer_gs = fig.add_gridspec(
      outer_rows,
      outer_cols,
      hspace=outer_hspace,
      wspace=outer_wspace,
      height_ratios=[1],
      width_ratios=[r, inner_aspect],
  )

  inner_gs = gridspec.GridSpecFromSubplotSpec(
      inner_rows,
      inner_cols,
      subplot_spec=outer_gs[1],
      wspace=inner_wspace,
      hspace=inner_hspace,
  )
  return (
      fig,
      outer_gs,
      inner_gs,
      inner_rows,
      inner_cols,
  )

In [ ]:
images_start, images_end, errors_start, errors_end, errors_start_l1, errors_end_l1 = load_images_and_errors()

crop_size = (350,200)
crop_size_1 = (192,110)
crop_size_1 = (106,60)
crop_size_2 = (248,141)
crop_size_3 = (248,141)

crop_sizes = [crop_size_1, crop_size_2, crop_size_3]

crop_offset_1 = (397, 622)
crop_offset_1 = (165, 581)

crop_offset_2 = (724, 232)
crop_offset_2 = (724, 504)

crop_offset_3 = (1057, 733)

crop_color_1 = "orange"
crop_color_2 = "green"
crop_color_3 = "purple"
crop_colors = [crop_color_1, crop_color_2, crop_color_3]
crop_offsets = [crop_offset_1, crop_offset_2, crop_offset_3]


image_ref = images_end[-1]
crop_images_end_1 = [figutils.crop_image(img,crop_offset_1,crop_size_1) for img in images_end]
crop_images_end_2 = [figutils.crop_image(img,crop_offset_2,crop_size_2) for img in images_end]
crop_images_end_3 = [figutils.crop_image(img,crop_offset_3,crop_size_3) for img in images_end]
all_crop_images = [crop_images_end_1,crop_images_end_2,crop_images_end_3]

In [ ]:
# @title Main figure

titles = [
    "Standard Adam",
    "Coarse to fine",
    "Mipmapping",
    "Coarse to fine\n+ Mipmapping",
    "Ours",
    "Reference",
]

test_ref_img = np.ones((1200, 1500, 3)) * np.linspace(0.3, 0.9, 1500)[:, None]
test_crop_img = (
    np.ones((crop_size[1], crop_size[0], 3))
    * np.linspace(0.3, 0.9, crop_size[0])[:, None]
)
line_width = 0.75
(
    fig,
    outer_gs,
    inner_gs,
    inner_rows,
    inner_cols,
) = figure_grid_setup(
    figutils.TEXT_WIDTH,
    test_ref_img.shape[:2],
    test_crop_img.shape[:2],
    inner_space=0.025,
    outer_space=0.005,
)

use_l2 = True

if True:
  ax = fig.add_subplot(outer_gs[0])
  ax.imshow(image_ref)
  figutils.disable_ticks(ax)
  ax.set_title(r"\textsc{The Art Gallery}", pad=6)
  label_l2 = (
      r"$L_2$ "
      + "error "
      + figutils.math_label(r"\text{$\times 10^3$}")
      + " (128 iterations) "
  )
  label_l1 = (
      r"$L_1$ "
      + "error "
      + figutils.math_label(r"\text{$\times 10^2$}")
      + " (128 iterations) "
  )
  ax.set_xlabel(label_l1 if not use_l2 else label_l2, labelpad=1.5)
  # add crops
  for crop_color, crop_offset,crop_size in zip(crop_colors, crop_offsets,crop_sizes):
    rect_closeup = Rectangle(
        crop_offset,
        crop_size[0],
        crop_size[1],
        linewidth=line_width,
        edgecolor=crop_color,
        facecolor="none",
    )
    ax.add_patch(rect_closeup)

errors = errors_start_l1 if not use_l2 else errors_start
for row in range(inner_rows):
  for col in range(inner_cols):
    is_reference = col == inner_cols - 1

    ax = fig.add_subplot(inner_gs[row, col])
    ax.imshow(all_crop_images[row][col])
    figutils.disable_ticks(ax)

    # Set Title
    if row == 0:
      ax.set_title(titles[col], pad=6 if col != 3 else 0)

    # Set image crops
    ax.spines[:].set_color(crop_colors[row])
    ax.spines[:].set_linewidth(line_width)

    # Set label
    if is_reference:
      label_l2 = ""
      label_l1 = ""
    elif col == 4:
      label_l2 = r"\textbf{" + f"{errors[col]*1000:.3f}" + r"}"
      label_l1 = r"\textbf{" + f"{errors[col]*100:.3f}" + r"}"
    else:
      label_l2 = f"{errors[col]*1000:.3f}"
      label_l1 = f"{errors[col]*100:.3f}"
    ax.set_xlabel(label_l1 if not use_l2 else label_l2, labelpad=1.5)


figutils.force_post_crop_size(fig, figutils.TEXT_WIDTH)

In [ ]:
figutils.savefig(
    fig,
    name=Path(FIGURE_NAME),
    fig_directory=Path(FIGURE_DIR),
    dpi=300,
    pad_inches=0.005,
    bbox_inches="tight",
    compress=False,
    target_width=figutils.TEXT_WIDTH,
    backend=None,
)